# Summary

This notebook calculates price growths within a specified horizon from every week and bins them into discrete levels, to be used as the multi-label target variable for a prediction task.

# Imports and configuration

In [1]:
import sys
import pickle
import numpy as np
import pandas as pd

from typing import List, Tuple

In [2]:
sys.path.append(r"C:\Users\mushj\OneDrive\Desktop\WORK\Financial Analytics\pipeline")

from fin_ml.data_engineering.discretize_price_growth import engineer_targets, generate_symmetric_intervals

In [3]:
INPUT_PATH = "C:/Users/mushj/Downloads/PROCESSED FINANCE DATA/FMP"
OUTPUT_PATH = "C:/Users/mushj/Downloads/CURATED FINANCE DATA/FMP"

In [4]:
# load historical daily prices
df = pd.read_csv(INPUT_PATH+'/FMP_daily_prices_top1k.csv')

# load list of symbols to keep (based on exploratory analysis)
with open(INPUT_PATH+'/top1k_subset', 'rb') as f:
    include_symbols = pickle.load(f)

# Prepare DataFrame

In [5]:
print("Number of symbols to include:", len(include_symbols))
print("Examples:", include_symbols[:5])

Number of symbols to include: 978
Examples: ['A', 'AA', 'AAL', 'AAON', 'AAP']


In [6]:
df = df.query("symbol in @include_symbols")
df['date'] = pd.to_datetime(df['date'])

In [7]:
df.head()

,symbol,date,close,volume
0,A,2005-01-03,16.09,3587208.0
1,A,2005-01-04,15.66,3978002.0
2,A,2005-01-05,15.66,4139634.0
3,A,2005-01-06,15.31,3353443.0
4,A,2005-01-07,15.30,2786175.0


# Engineer multi-label target variable

1. The daily stock price data is downsampled to a weekly frequency. The last daily closing price is used as the week's closing price. For example, if the week contains prices for Monday till Friday, then Friday's closing price is used.
2. Given a forecast horizon (e.g. 1 year ahead), price growths are computed for each week, between a week's closing price and EVERY subsequent DAILY closing price within the forecast horizon.
3. Discrete intervals are defined, for example, <-25%, -25to0%, 0to25% >25%. The multi-label target variable for each week is a vector of binary values that indicate whether a stock will experience a certain level of price growth within the given horizon from that week. For example, a vector of [0, 1, 0, 1] for week 50 of stock ABC means that the stock will experience at least one price change in the interval -25to0%, and at least one price change in the interval >25%, relative to the closing price of week 50.

## Get weekly data

In [8]:
%%time
weekly_df = (
    df
    .sort_values(['symbol', 'date'])
    .groupby('symbol')
    .resample('W-MON', on='date', label='left') # mark each week with Monday
    .last() # get last price of the week
    .reset_index(level='date') # set date index as column
    .reset_index(drop=True) # remove other indexes
    .rename({'date': 'week'}, axis=1)
)

CPU times: total: 19.5 s
Wall time: 19.8 s


In [9]:
weekly_df.head()

,week,symbol,close,volume
0,2004-12-27,A,16.09,3587208.0
1,2005-01-03,A,15.23,4190098.0
2,2005-01-10,A,14.86,3027509.0
3,2005-01-17,A,14.45,5241370.0
4,2005-01-24,A,14.90,2821500.0


In [10]:
weekly_df.isna().mean()

week      0.0
symbol    0.0
close     0.0
volume    0.0
dtype: float64

In [11]:
weekly_df.dtypes

week      datetime64[ns]
symbol            object
close            float64
volume           float64
dtype: object

In [47]:
# distribution of counts of weekly data points
weekly_df.value_counts('symbol').describe()

count     978.000000
mean      871.982618
std       282.940932
min       155.000000
25%       709.250000
50%      1045.000000
75%      1045.000000
max      1045.000000
Name: count, dtype: float64

## Validate weekly data

In [12]:
%%time
# store the complete sets of weeks between the start and end weeks of each symbol (span)
week_span = {}

for symbol, data in weekly_df.groupby('symbol'):
    weeks = data.query("symbol == @symbol")['week']
    start, end = weeks.min(), weeks.max()
    week_span[symbol] = set(pd.date_range(start, end, freq='W-MON'))

CPU times: total: 20.6 s
Wall time: 21.1 s


In [13]:
%%time
# detect missing weeks for each symbol
missing_weeks = []

for symbol, data in weekly_df.groupby('symbol'):
    # get weeks in symbol span that are not in symbol data
    missing = week_span[symbol].difference(set(data['week']))
    for w in missing:
        missing_weeks.append({'symbol': symbol, 'missing_week': w})
        
missing_weeks = pd.DataFrame(missing_weeks)

CPU times: total: 1.38 s
Wall time: 1.39 s


In [14]:
# check missing weeks
missing_weeks

""


## Set intervals

In [15]:
# log-spaced price changes between 1% and 15%
boundaries = np.logspace(np.log(0.02), np.log(0.15), num=4, base=np.e)
np.round(boundaries, 3)

array([0.02 , 0.039, 0.077, 0.15 ])

In [16]:
boundaries = [0, 0.02, 0.039, 0.077, 0.15, np.inf]
intervals = generate_symmetric_intervals(boundaries)

print(len(intervals))
for i in intervals:
    print(i)

10
(-inf, -0.15)
(-0.15, -0.077)
(-0.077, -0.039)
(-0.039, -0.02)
(-0.02, 0)
(0, 0.02)
(0.02, 0.039)
(0.039, 0.077)
(0.077, 0.15)
(0.15, inf)


## Compute price changes - 30 days

In [17]:
weekly_df = weekly_df.rename({'week': 'date'}, axis=1)

In [43]:
%%time
# calculate for each symbol separately
for symbol, data in weekly_df.groupby('symbol'):
    target_30d_df = engineer_targets(
        data, df, 
        forecast_horizon=30, horizon_margin=5, intervals=intervals
    )
    target_30d_df.to_csv(OUTPUT_PATH+f'/discrete_return_30d/{symbol}.csv', index=False)

Calculating for BFAM: 100%|█████████████████████████████████████████████████████████| 624/624 [00:02<00:00, 219.59it/s]


Calculating for CHTR: 100%|█████████████████████████████████████████████████████████| 783/783 [00:03<00:00, 218.83it/s]


Calculating for DASH: 100%|█████████████████████████████████████████████████████████| 213/213 [00:01<00:00, 166.42it/s]


Calculating for EQT: 100%|████████████████████████████████████████████████████████| 1045/1045 [00:06<00:00, 173.50it/s]


Calculating for GIS: 100%|████████████████████████████████████████████████████████| 1045/1045 [00:05<00:00, 178.23it/s]


...

Calculating for SLM: 100%|████████████████████████████████████████████████████████| 1045/1045 [00:04<00:00, 224.22it/s]


Calculating for TRIP: 100%|█████████████████████████████████████████████████████████| 683/683 [00:03<00:00, 224.70it/s]


Calculating for WBS: 100%|████████████████████████████████████████████████████████| 1045/1045 [00:04<00:00, 220.75it/s]


Calculating for ZTS: 100%|██████████████████████████████████████████████████████████| 623/623 [00:02<00:00, 226.11it/s]


CPU times: total: 6min 29s
Wall time: 1h 11min 38s


In [49]:
with open(OUTPUT_PATH+f'/discrete_return_30d/labels', 'wb') as f:
    pickle.dump(intervals, f)

## Sample result

In [51]:
target_30d_df

,symbol,week,labels
0,ZTS,2013-01-28,"[0, 0, 0, 0, 1, 1, 1, 1, 1, 0]"
1,ZTS,2013-02-04,"[0, 0, 1, 1, 1, 1, 1, 1, 0, 0]"
2,ZTS,2013-02-11,"[0, 0, 1, 1, 1, 1, 0, 0, 0, 0]"
3,ZTS,2013-02-18,"[0, 0, 0, 0, 1, 1, 1, 1, 1, 0]"
4,ZTS,2013-02-25,"[0, 0, 1, 1, 1, 1, 0, 0, 0, 0]"
...,...,...,...
618,ZTS,2024-12-02,"[0, 1, 1, 0, 1, 1, 0, 0, 0, 0]"
619,ZTS,2024-12-09,NaN
620,ZTS,2024-12-16,NaN
621,ZTS,2024-12-23,NaN


In [44]:
t_df = pd.DataFrame(target_30d_df.dropna()['labels'].to_list())
t_df.columns = intervals

In [45]:
t_df.describe()

,"(-inf, -0.15)","(-0.15, -0.077)","(-0.077, -0.039)","(-0.039, -0.02)","(-0.02, 0)","(0, 0.02)","(0.02, 0.039)","(0.039, 0.077)","(0.077, 0.15)","(0.15, inf)"
count,619.000000,619.000000,619.000000,619.000000,619.0,619.000000,619.000000,619.000000,619.000000,619.000000
mean,0.009693,0.124394,0.368336,0.646204,1.0,0.943457,0.689822,0.458805,0.156704,0.012924
std,0.098054,0.330298,0.482743,0.478534,0.0,0.231154,0.462940,0.498703,0.363816,0.113038
min,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,1.0,1.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000,1.0,1.000000,1.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
